# Unigram Model 
---

In [2]:
from nltk.corpus import stopwords as sw
from nltk.tokenize import word_tokenize as wt
from nltk import PorterStemmer  
import re
from nltk import FreqDist
from nltk.probability import LidstoneProbDist, WittenBellProbDist
import sys
from IPython.display import display
#sys.stdout = stdout
#reload(sys)
#sys.setdefaultencoding("utf-8")
stopWord = set(sw.words("english"));
import numpy as np
import pandas as pds
import math
import os
from __future__ import print_function

# variables
---

In [3]:
files = ["ray_913742943690653739_cf.txt", "ray_925987435331224445_cf.txt", 
        "ray_925987537478928034_pf.txt"]
lam = 0.3
topNRDoc = 10;

# [Functions]

# function to clean document 
---

In [4]:
# getting word from documents

In [5]:
#cleaning the words

In [6]:
dots = ['.', ',', '!', '', "+", "#", "(", ")", ":", "'s"]
slangs = {"n't":"not", "r": "are", "u": "you"}
def goclean(doc):
    doc = doc.lower()
    #\W+
    tokens = wt(doc)
    filterWord = [];
    
    for w in tokens:
        if w not in dots and w not in stopWord:
            if w in slangs:
                w = slangs[w];
            filterWord.append(w)
    sents = " ".join(filterWord)
    filterWord = re.findall('\w+', sents)
    ps = PorterStemmer()
    stemed = []
    for w in filterWord:
        fword = ps.stem(w)
        stemed.append(fword) 
    return stemed

# function to calculate probability
---

In [7]:
def getprobability(cleanW):
    myarray = np.asarray(cleanW)
    ue, ce = np.unique(myarray, return_counts=True)
    tc = len(cleanW);
    n = float(tc)
    
    p = [];
    for i in range(len(ce)): 
        p.append(ce[i]/n)
    
    mydict = dict(zip(ue, p))
    return mydict

In [8]:
#smoothing function not in use now

In [9]:
def smooth(probd, probc, lam):
    dict2 = dict(probc)
    
    for key, items in probc.iteritems():
        if key in probd:
            val = (1 - lam)*probd[key] + lam*probc[key];
            #pdf[name][key] = val;
            dict2[key] = val
        
        else:
            val = lam*(probc[key]);
            #pdf[name][key] = val;
            dict2[key] = val
    return dict2;

In [10]:
#change in formula instead of smoothing we get prob ratio

In [11]:
def getpratio(probd, probc, lam):
    dict2 = dict(probc)
    
    for key, items in probc.iteritems():
        if key in probd:
            val = ((1 - lam)*probd[key])/(lam*probc[key]) + 1;
            #pdf[name][key] = val;
            dict2[key] = val
        
        else:
            val = 1
            #pdf[name][key] = val;
            dict2[key] = val
    return dict2;

# inverse indexing
---

In [12]:
# mapping words to document

In [13]:
def getWordMap(clrev, wordMap, i):
    for key in clrev:
        wordMap[key].append(i)
    return wordMap

In [14]:
#merging wordMap for a query to get the list of doc which contain the words

In [15]:
def getMerge(wordMap, word):
    docList = []
    for w in word:
        if w not in pdf.index:
            continue;
        docList= list(set(docList + wordMap[w]))
    return docList

# KL divergence / Ranking function
---

In [16]:
def KLdivergence(q, df, docList):
    TD = len(docList)
    x = [0]*TD;
    for word, prob in q.iteritems():
        if word not in pdf.index:
            continue;
        for i in range(TD):
            name = 'probd' + str(docList[i])
            x[i] = x[i] + prob*(math.log(pdf[name][word]))
    return x;       

# working on collection
---

In [17]:
doc = ""
def getdoc(files, doc):
    for f in files:
        doc += open(f , 'r').read()
    return doc

doc += getdoc(files, doc);
doc = doc.decode('utf-8');
cw = goclean(doc)

In [18]:
probc = getprobability(cw)
dict2 = dict(probc)
pdf = pds.DataFrame(dict2.values(), index=dict2.keys(), columns=["probc"])

In [19]:
tc = len(probc)
values = [[] for _ in range(tc)]
keys = dict2.keys()
wordMap = dict(zip(keys, values)) 

# Working on documents / Doc model
---

In [20]:
reviews = doc.split("\n")
td = len(reviews)
for i in range(len(reviews)):
    name = "probd" + str(i);
    
    clrev = goclean(reviews[i]) # ['what', 'should', 'not', 'be', 'done'] without removing stopword
    
    wordMap = getWordMap(clrev, wordMap, i) #{u'andra': [1084, 1084], u'authorit': [254, 255, 784, 254, 255, 784],..}
    
    prob = getprobability(clrev) #{'be': 0.20000000000000001,'done': 0.20000000000000001,........}
    
    ps = getpratio(prob, probc, lam) #((1 - lam)*probd[key])/(lam*probc[key]) + 1;
    
    pdf[name] = ps.values() #including the doc model in data frame

# query modeling
---

In [21]:
#q = (reviews[16] + '.')[:-1]
q = raw_input() # "what shouldn't be done"

cq = goclean(q) # ['what', 'should', 'not', 'be', 'done'] without removing stopword

qprob = getprobability(cq) #{'be': 0.20000000000000001,'done': 0.20000000000000001,........}  

docList = getMerge(wordMap, cq) #[1195, 4, 519, 522, 524, .......] list containg words in query

div = KLdivergence(qprob, pdf, docList) #ranking function using KL divergens

index = div.index(max(div)) #most probabel relevent doc

#reviews[docList[index]]

good work


# top N Relevent doc
---

In [22]:
tlist = div[:];
x = []#*topNRDoc

for i in range(min(topNRDoc, len(tlist))):
    Pmax = max(tlist)
    #if least == 0:
        #break
    index = tlist.index(Pmax);
    x.append(docList[index])
    tlist[index] = -1;

    
rdl = len(x)
if rdl == 0:
    print("No relevent document found")
    
else:
    prt = ""
    for i in range(rdl):
        display(str(i+1) + ') ' + reviews[x[i]])

u'1) good work JC'

u'2) good strong, engaging pitch. Good work here'

u'3) Nice work. Good use of customer examples.'

u'4) Good composure and good flow. Good speech avoiding \u201c\u2026euh\u2026\u201d during the messaging. Good work on the eye contact. Case studies are well presented.'

u'5) Hit the elements of the solutions and strategy. Good work including customer references.'

u'6) Good work on the second attempt. You kept it concise and you hit on the primary differentiators.'

u'7) Hi Piyabut, good work in going through this and coming out as a clear and detailed presenter!'

u'8) Like the affiliation with a specific customer, good work going off-script.'

u'9) Good work Chalita! I like the way you asked the question to start the pitch.'

u'10) Good work Vincent! Like the way you pace it and how you go down the different layers.'